# Estimation 

In this notebook, we construct a giant likelihood with price, viewership, and sampling weights and estimate everything out. 

In [21]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

In [22]:
%%stata -s nh1
clear all
cd C:\Users\mjbaker\Documents\Github\NewsHour
use AveragedDataDyno.dta
set more off
set seed 8675309
sort market stationid mt timeslot


C:\Users\mjbaker\Documents\Github\NewsHour


## Step 1 - read in some data and organize it.

Here, we have to organize data and get everything going. Really, this is the same stuff we did in notebook one to get dynamic data aligned with the (observed) viewership likelihood. ALong these lines, we need to reconstitute a few variables that we use and get them into mata. We will create a marker variable so that things work. 

While previously we developed a "game" variable in `mata`, this time around I'll do it in `Stata` just to see if we get the same result:

In [23]:
%%stata -s nh1
bysort stationid: egen countl = total(lnews)
bysort stationid: gen lasts = _n == _N

gen game=lnpps!=. & (lnews | otherl)
replace game = 0 if countl ==0


(1328 real changes made)


In [24]:
%%stata -s nh1
gen useOrd =_n
tsset stationid timeslot

gen otherlPrime=lnews&game
gen lnewsPrime =otherl&game
gen nnewsPrime =nnews&game

gen lnewslnewsPrime=otherl*l.lnews
gen lnewsnnewsPrime=otherl*l.nnews
gen nnewslnewsPrime=nnews*l.lnews
gen nnewsnnewsPrime=nnews*l.nnews
gen lsiPrime=l.si
replace lsiPrime=ln(lsiPrime)
replace lsiPrime=0 if lsiPrime==.

gen siXlnlnPrime=lnewslnewsPrime*lsi
gen siXlnnnPrime=lnewsnnewsPrime*lsi
gen siXnnlnPrime=nnewslnewsPrime*lsi
gen siXnnnnPrime=nnewsnnewsPrime*lsi
gen lnewstotPrime=otherl*ln(1+totslnews)
gen nnewstotPrime=nnews*ln(1+totsnnews)
sort useOrd 


       panel variable:  stationid (strongly balanced)
        time variable:  timeslot, 3 to 8
                delta:  1 unit
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(83110 real changes made)
(16622 real changes made)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)


Pull all the variables into `mata`:

In [25]:
%%stata -s nh1
mata:
    st_view(lnewsPrime=.,.,"lnewsPrime")
    st_view(otherlPrime=.,.,"otherlPrime")
    st_view(nnewsPrime=.,.,"nnewsPrime")
    st_view(lnewslnewsPrime=.,.,"lnewslnewsPrime")
    st_view(lnewsnnewsPrime=.,.,"lnewsnnewsPrime")
    st_view(nnewslnewsPrime=.,.,"nnewslnewsPrime")
    st_view(nnewsnnewsPrime=.,.,"nnewsnnewsPrime")
    st_view(lsiPrime=.,.,"lsiPrime")
    st_view(siXlnlnPrime=.,.,"siXlnlnPrime")
    st_view(siXlnnnPrime=.,.,"siXlnnnPrime")
    st_view(siXnnlnPrime=.,.,"siXnnlnPrime")
    st_view(siXnnnnPrime=.,.,"siXnnnnPrime")
    st_view(lnewstotPrime=.,.,"lnewstotPrime")
    st_view(nnewstotPrime=.,.,"nnewstotPrime")

    st_view(lnewslnews=.,.,"lnewslnews")
    st_view(lnewsnnews=.,.,"lnewsnnews")
    st_view(nnewslnews=.,.,"nnewslnews")
    st_view(nnewsnnews=.,.,"nnewsnnews")
    st_view(lsi=.,.,"lsi")
    st_view(siXlnln=.,.,"siXlnln")
    st_view(siXlnnn=.,.,"siXlnnn")
    st_view(siXnnln=.,.,"siXnnln")
    st_view(siXnnnn=.,.,"siXnnnn")
    st_view(lnewstot=.,.,"lnewstot")
    st_view(nnewstot=.,.,"nnewstot")
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(othercn=.,.,"othercn")
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     st_view(lnewsPrime=.,.,"lnewsPrime")

:     st_view(otherlPrime=.,.,"otherlPrime")

:     st_view(nnewsPrime=.,.,"nnewsPrime")

:     st_view(lnewslnewsPrime=.,.,"lnewslnewsPrime")

:     st_view(lnewsnnewsPrime=.,.,"lnewsnnewsPrime")

:     st_view(nnewslnewsPrime=.,.,"nnewslnewsPrime")

:     st_view(nnewsnnewsPrime=.,.,"nnewsnnewsPrime")

:     st_view(lsiPrime=.,.,"lsiPrime")

:     st_view(siXlnlnPrime=.,.,"siXlnlnPrime")

:     st_view(siXlnnnPrime=.,.,"siXlnnnPrime")

:     st_view(siXnnlnPrime=.,.,"siXnnlnPrime")

:     st_view(siXnnnnPrime=.,.,"siXnnnnPrime")

:     st_view(lnewstotPrime=.,.,"lnewstotPrime")

:     st_view(nnewstotPrime=.,.,"nnewstotPrime")

:     st_view(lnewsnnews=.,.,"lnewsnnews")

:     st_view(nnewslnews=.,.,"nnewslnews")

:     st_view(nnewsnnews=.,.,"nnewsnnews")

:     st_view(lsi=.

To make it easier to pass things around, let's create a structured object and use it to pass all this stuff around. We also make a simple initiation function to deal with it (this would be a bit simpler in Python!)

In [26]:
%%stata -s nh1
mata: 
    struct dynoInfo {
        real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews, lsi,
            siXlnln, siXlnnn, siXnnln, siXnnnn, lnewstot, nnewstot, l_ACS_HH,
            lnewsn,otherln,nnewsn,othercn
    }
    
    struct dynoInfo dynoInfoInit(real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,lnewsn,otherln,nnewsn,othercn)
    {
        struct dynoInfo scalar Dy
    
        Dy.lnewslnews=lnewslnews
        Dy.lnewsnnews=lnewsnnews
        Dy.nnewslnews=nnewslnews
        Dy.nnewsnnews=nnewsnnews
        Dy.lsi=lsi
        Dy.siXlnln=siXlnln
        Dy.siXlnnn=siXlnnn
        Dy.siXnnln=siXnnln
        Dy.siXnnnn=siXnnnn
        Dy.lnewstot=lnewstot
        Dy.nnewstot=nnewstot
        Dy.l_ACS_HH=l_ACS_HH
        Dy.lnewsn=lnewsn
        Dy.otherln=otherln
        Dy.nnewsn=nnewsn
        Dy.othercn=othercn
        return(Dy)
    }
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     struct dynoInfo {
>         real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews, lsi,
>             siXlnln, siXlnnn, siXnnln, siXnnnn, lnewstot, nnewstot, l_ACS_HH,
>             lnewsn,otherln,nnewsn,othercn
>     }

:     
:     struct dynoInfo dynoInfoInit(real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
>         lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,lnewsn,otherln,nnewsn,othercn)
>     {
>         struct dynoInfo scalar Dy
>     
>         Dy.lnewslnews=lnewslnews
>         Dy.lnewsnnews=lnewsnnews
>         Dy.nnewslnews=nnewslnews
>         Dy.nnewsnnews=nnewsnnews
>         Dy.lsi=lsi
>         Dy.siXlnln=siXlnln
>         Dy.siXlnnn=siXlnnn
>         Dy.siXnnln=siXnnln
>         Dy.siXnnnn=siXnnnn
>         Dy.lnewstot=lnewstot
>         Dy.nnewstot=nnewstot
>

Now, just pass all the information along for use in estimation.

In [27]:
%%stata -s nh1
mata:
    Dy=dynoInfoInit(lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,
        lnewsn,otherln,nnewsn,othercn)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     Dy=dynoInfoInit(lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
>         lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,
>         lnewsn,otherln,nnewsn,othercn)

: end


## Price information

Since the above is a pretty convenient way of dealing with a lot of variables, let's also use our previously constructed and saved data on pricing, and pack it up in a similar way:

In [28]:
%%stata -s nh1
mata:
    mata matuse MataPriceData
end
    


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matuse MataPriceData
(loading idt[3738,1], l_ACS_HHt[3738,1], lnewst[3738,1], lnppst[3738,1], lnviewt[3738,1], mat[101,2], mtt[3738,1], nnewst[3738,1], otherlt[3738,1])

: end


A structured object containing all this information, and using it to pass stuff along:

In [29]:
%%stata -s nh1
mata:
   struct priceInfo {
        real matrix y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m
    }
    struct priceInfo priceInfoInit(y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m)
    {
        struct priceInfo scalar pInfo
    
        pInfo.y=y
        pInfo.lnewst=lnewst
        pInfo.otherlt=otherlt
        pInfo.nnewst=nnewst
        pInfo.lnviewt=lnviewt
        pInfo.id=id
        pInfo.mt=mt
        pInfo.m=m
        pInfo.l_ACS_HHt=l_ACS_HHt
    return(pInfo)
    }
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:    struct priceInfo {
>         real matrix y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m
>     }

:     struct priceInfo priceInfoInit(y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m)
>     {
>         struct priceInfo scalar pInfo
>     
>         pInfo.y=y
>         pInfo.lnewst=lnewst
>         pInfo.otherlt=otherlt
>         pInfo.nnewst=nnewst
>         pInfo.lnviewt=lnviewt
>         pInfo.id=id
>         pInfo.mt=mt
>         pInfo.m=m
>         pInfo.l_ACS_HHt=l_ACS_HHt
>     return(pInfo)
>     }

: end


Import our information on selection stuff and the game and all that:

In [30]:
%%stata -s nh1
mata:
    mata matuse gsAndus, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matuse gsAndus, replace
(loading NashW[99732,20], Up[99732,20], Upb[99732,20], Upg[99732,20], Upre1[99732,20], Upre1g[99732,20], Upre2[99732,20], Upre2g[99732,20], Uv[99732,20],
 Uvg[99732,20], Uvre1[99732,20], Uvre1g[99732,20], Uvre2[99732,20], Uvre2g[99732,20], Uvsg[99732,20], Uvsi[99732,20], Uvso[99732,20], Vup[99732,20], id[99732,1],
 market[99732,1], mt[99732,1], weights1[99732,1], weights2[99732,1])

: end


In [32]:
%%stata -s nh1
mata:
    st_view(game=.,.,"game")
    sum(NashW:!=.)
    sum(select(NashW,game):!=.)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     st_view(game=.,.,"game")

:     sum(NashW:!=.)
  2020

:     sum(select(NashW,game):!=.)
  0

: end


In [20]:
%%stata 
close all


Terminated 2 running Stata processes
